# Mini-Project: SVM & Logistic Regression Classification

Matt Farrow, Amber Clark, Blake Freeman, Megan Ball

## **2015 Flight Delays and Cancellations**
Data Source: [Kaggle](https://www.kaggle.com/usdot/flight-delays?select=flights.csv)

## Logistic Regression & Support Vector Machine Models

[50 points] Create a logistic regression model and a support vector machine model for the
classification task involved with your dataset. Assess how well each model performs (use 80/20 training/testing split for your data). Adjust parameters of the models to make them more accurate. If your dataset size requires the use of stochastic gradient descent, then linear kernel only is fine to use.

### Prep Data

#### Setup

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import sklearn as sk

# Import time-based columns as strings to keep leading zeroes
dtype_t = {'SCHEDULED_DEPARTURE': str,
           'DEPARTURE_TIME': str,
           'WHEELS_OFF': str,
           'SCHEDULED_TIME': str,
           'WHEELS_ON': str,
           'SCHEDULED_ARRIVAL': str,
           'ARRIVAL_TIME': str
          }

#### Read & Merge Data

In [2]:
# Read in the data directly using Pandas
airlines = pd.read_csv('../Data/airlines.csv')
airports = pd.read_csv('../Data/airports.csv')
flights  = pd.read_csv('../Data/flights.csv', dtype = dtype_t)

# Read in the data directly from GitHub
# airlines = pd.read_csv('https://raw.githubusercontent.com/mattfarrow1/7331-machine-learning-1/main/Data/airlines.csv')
# airports = pd.read_csv('https://raw.githubusercontent.com/mattfarrow1/7331-machine-learning-1/main/Data/airports.csv')
# flights  = pd.read_csv('https://media.githubusercontent.com/media/mattfarrow1/7331-machine-learning-1/main/Data/flights.csv', dtype = dtype_t)

# Rename columns in preparation for merge
airlines.rename(columns={'IATA_CODE': 'AIRLINE_CODE'}, inplace=True)
flights.rename(columns={'AIRLINE': 'AIRLINE_CODE'}, inplace=True)

# Merge data together
df = pd.merge(flights, airlines, on='AIRLINE_CODE', how = 'left')

/Users/mattfarrow/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Format Date & Time-Based Columns

In [3]:
# Convert string columns to datetime
cols = ["SCHEDULED_DEPARTURE", 
        "DEPARTURE_TIME", 
        "WHEELS_OFF",  
        "WHEELS_ON", 
        "SCHEDULED_ARRIVAL", 
        "ARRIVAL_TIME"]

# Coerce time-based columns to an Hour:Minute format
df[cols] = df[cols].apply(pd.to_datetime, format = '%H%M', errors='coerce')

# Convert YMD into a single date
# Source: https://stackoverflow.com/questions/54487059/pandas-how-to-create-a-single-date-column-from-columns-containing-year-month
df['FLIGHT_DATE'] = pd.to_datetime([f'{y}-{m}-{d}' for y, m, d in zip(df.YEAR, df.MONTH, df.DAY)])

#### Exclude Cancelled Flights

In [4]:
df = df[df['CANCELLED'] == 0]

#### Deal with Missing Values

In [5]:
# Fill missing delay values with 0
cols = ['AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']
df[cols] = df[cols].fillna(0)

# Drop unnecessary columns: 
#    CANCELLATION_REASON since all cancelled flights are gone
#    YEAR, MONTH, DAY since those have been converted to a single date already
#    AIRLINE_CODE since AIRLINE has been merged in
#    SCHEDULED_TIME since that's still an object
df = df.drop(['YEAR', 'MONTH', 'DAY', 'CANCELLED', 'CANCELLATION_REASON', 'AIRLINE_CODE', 'SCHEDULED_TIME'], axis=1)

# There are 15,187 remaining rows with missing values for ARRIVAL_DELAY. We'll remove remaining null values.
df = df.dropna()

#### Bucket Data

In [6]:
# Flight Distance
distance_labels = ['Short', 'Medium', 'Long']
distance_bins   = [1, 100, 1000, np.inf]
df['DISTANCE_BUCKET'] = pd.cut(df['DISTANCE'],
                               bins=distance_bins,
                               labels=distance_labels)

# Arrival Delay
delay_labels = ['Early', 'On-Time', '30-59', '60-89', '90-119', '120+']
delay_bins   = [-np.inf, 0, 30, 60, 90, 120, np.inf]
df['ARRIVAL_DELAY_BUCKET'] = pd.cut(df['ARRIVAL_DELAY'],
                                    bins=delay_bins,
                                    labels=delay_labels)

#### Create Response Variable

In [7]:
# Create a new column where the arrival_delay > 0 means it's delayed(=1) and if <= 0 it's not delayed(=0)
get_delay = lambda x: 0 if x <= 0 else 1
df['DELAYED'] = df.ARRIVAL_DELAY.apply(get_delay)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5708334 entries, 0 to 5819078
Data columns (total 29 columns):
 #   Column                Dtype         
---  ------                -----         
 0   DAY_OF_WEEK           int64         
 1   FLIGHT_NUMBER         int64         
 2   TAIL_NUMBER           object        
 3   ORIGIN_AIRPORT        object        
 4   DESTINATION_AIRPORT   object        
 5   SCHEDULED_DEPARTURE   datetime64[ns]
 6   DEPARTURE_TIME        datetime64[ns]
 7   DEPARTURE_DELAY       float64       
 8   TAXI_OUT              float64       
 9   WHEELS_OFF            datetime64[ns]
 10  ELAPSED_TIME          float64       
 11  AIR_TIME              float64       
 12  DISTANCE              int64         
 13  WHEELS_ON             datetime64[ns]
 14  TAXI_IN               float64       
 15  SCHEDULED_ARRIVAL     datetime64[ns]
 16  ARRIVAL_TIME          datetime64[ns]
 17  ARRIVAL_DELAY         float64       
 18  DIVERTED              int64         
 19  

#### One-Hot Encoding

In [9]:
# Source: datagy.io/pandas-get-dummies/

one_hot_columns = ['TAIL_NUMBER',
                   'ORIGIN_AIRPORT', 
                   'DESTINATION_AIRPORT',
                   'AIRLINE',
                  'DISTANCE_BUCKET',
                  'ARRIVAL_DELAY_BUCKET']

for column in one_hot_columns:
    tempdf = pd.get_dummies(df[column], prefix=column)
    
    df = pd.merge(
        left = df,
        right = tempdf,
        left_index = True,
        right_index = True
    )

In [10]:
df.shape

(5708334, 6806)

#### Sample 1% of Records

In [11]:
df_sample = df.sample(frac=0.01)

In [12]:
df_sample.shape

(57083, 6806)

#### Test/Train Split

In [13]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold

# Split response variable from rest of the data:
if 'DELAYED' in df_sample:
    y = df_sample['DELAYED'].values # get the labels we want
    del df_sample['DELAYED']        # get rid of the class label
    X = df_sample.values            # use everything else to predict

# Split using cross validation
num_cv_iterations = 1
num_instances = len(y)
cv_object = ShuffleSplit(n_splits=num_cv_iterations, test_size  = 0.2, random_state = 42)
# cv_object = StratifiedKFold(n_splits = num_cv_iterations, shuffle = True, random_state = 42)

print(cv_object)

ShuffleSplit(n_splits=1, random_state=42, test_size=0.2, train_size=None)


In [14]:
# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in cv_object.split(X,y): 
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]

In [15]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((45666, 6805), (11417, 6805), (45666,), (11417,))

#### Create SVM Model

In [16]:
%%time
from sklearn.svm import SVC

#Create a Linear regression object and perform a grid search to find the best parameters
reg = SVC()

#Set up SVC parameters to test 
costs = [0.01, 3.0]
gamma = ['scale','auto']
kernels = ['poly', 'rbf', 'sigmoid']
parameters = {'C': costs, 'gamma' : gamma, 'kernel': kernels}

#Create a grid search object using the parameters above
from sklearn.model_selection import GridSearchCV
svmGridSearch = GridSearchCV(estimator=reg,
                             n_jobs=-1, # automatically use every available core
                             verbose=1, # low verbosity
                             param_grid=parameters,
                             cv=cv_object,
                             scoring='accuracy')

#Perform hyperparameter search to find the best combination of parameters for our data
svmGridSearch.fit(X_train, y_train)

Fitting 1 folds for each of 12 candidates, totalling 12 fits


/Users/mattfarrow/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  category=UserWarning


ValueError: could not convert string to float: 'N882AS'

In [ ]:
# Display best estimator parameters
svmGridSearch.best_estimator_

In [ ]:
%%time
# Use the best object
classifierEst2 = svmGridSearch.best_estimator_

# Evaluate the SVM estimator above using pre-defined cross validation and scoring metrics. 
EvaluateClassifierEstimator(classifierEst2, X_train, y_train, cv_object)

## Advantages of Each Model

[10 points] Discuss the advantages of each model for each classification task. Does one type of model offer superior performance over another in terms of prediction accuracy? In terms of training time or efficiency? Explain in detail.

## Feature Importance

[30 points] Use the weights from logistic regression to interpret the importance of different features for each classification task. Explain your interpretation in detail. Why do you think some variables are more important?

## Support Vectors

[10 points] Look at the chosen support vectors for the classification task. Do these provide any insight into the data? Explain.